Structure:
    
- get some food data
- make food object, bare classes showing dunder (magic) methods (init, str/repr, compare, add)
- food log class to show more dunders: (len, iter, create a context manager)
- create some functions using the food objects
- add exception handling (raise, catch, own exception, best practices)

In [1]:
file = "calories.csv"  # https://github.com/PythonCharmers/PythonCharmersData/blob/master/calories.csv

In [2]:
import csv

In [4]:
with open(file) as f:
    for row in csv.DictReader(f):
        print(row)

FileNotFoundError: [Errno 2] No such file or directory: 'calories.csv'

In [5]:
from functools import total_ordering


@total_ordering
class Food:
    def __init__(self, name, measure, weight, kcal):
        self.name = name
        self.measure = measure
        self.weight = weight
        self.kcal = kcal
    
    def __repr__(self):
        cls_name = type(self).__name__
        return f"{cls_name}('{self.name}', '{self.measure}', {self.weight}, {self.kcal})"
    
    def __str__(self):
        return self.name
    
    def __eq__(self, other):
        return self.kcal == other.kcal
    
    def __lt__(self, other):
        return self.kcal < other.kcal
    
    def __add__(self, other):          
        name = f"{self.name} & {other.name}"
        
        if self.measure != other.measure:
            raise ValueError("Measures should be the same")
            
        weight = self.weight + other.weight
        kcal = self.kcal + other.kcal
        
        cls = type(self)
        return cls(name, self.measure, weight, kcal)
    

In [6]:
pie = Food('Apple Pie', '100 grams', 158, 405)

In [7]:
bananas = Food("Bananas", "1 Piece", 114, 105)

In [8]:
bananas > pie

False

In [9]:
banana_apple_pie = bananas + pie

ValueError: Measures should be the same

In [10]:
class Car:
    pass

car = Car()

In [12]:
bananas + car  # question regarding adding different objects

AttributeError: 'Car' object has no attribute 'name'

In [13]:
# dataclasses save a lot of boilerplate code

from dataclasses import dataclass


@dataclass
class Food:
    name: str
    measure: str
    weight: int
    kcal: int
        
    def __str__(self):
        return self.name

    def __eq__(self, other):
        return self.kcal == other.kcal
    
    def __lt__(self, other):
        return self.kcal < other.kcal
    
    def __add__(self, other):          
        name = f"{self.name} & {other.name}"
        
        if self.measure != other.measure:
            raise ValueError("Measures should be the same")
            
        weight = self.weight + other.weight
        kcal = self.kcal + other.kcal
        
        cls = type(self)
        return cls(name, self.measure, weight, kcal)

In [14]:
pie = Food('Apple Pie', '100 grams', 158, 405)

In [15]:
bananas = Food("Bananas", "1 Piece", 114, 105)

In [16]:
pie > bananas

True

In [18]:
# showing some more dunder methods (and properties)

from dataclasses import field

@dataclass
class FoodLog:
    owner: str
    kcal_limit: int
    _foods: list[Food] = field(default_factory=list)
    index: int = 0
        
    def add_food(self, food: Food):
        self._foods.append(food)
    
    @property
    def total_calories_consumed(self):
        return sum(food.kcal for food in self._foods)
    
    @property
    def calories_to_consume_today(self):
        return self.kcal_limit - self.total_calories_consumed
    
    def __len__(self):
        return len(self._foods)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index < len(self._foods):
            self.index += 1
            return self._foods[self.index - 1]
        else:
            raise StopIteration

In [19]:
tracker = FoodLog("bob", 2500)

In [20]:
tracker.add_food(pie)
tracker.add_food(pie)
tracker.add_food(bananas)

In [21]:
tracker._foods

[Food(name='Apple Pie', measure='100 grams', weight=158, kcal=405),
 Food(name='Apple Pie', measure='100 grams', weight=158, kcal=405),
 Food(name='Bananas', measure='1 Piece', weight=114, kcal=105)]

In [22]:
len(tracker)

3

In [23]:
for food in tracker: print(food)

Apple Pie
Apple Pie
Bananas


In [24]:
next(tracker)

StopIteration: 

TODO / homework, could write a context manager that lets you add a food only when you have calories left. Or could just add regular exception handling to `FoodLog`'s `add_food` method ...